In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import logging

import scipy.stats
import numpy as np
import pandas as pd

In [2]:
import diffxpy.api as de

# Introduction

Perfect confounding occurs frequently in differential expression assays, often if biological replicates cannot be spread across conditions: This is often the case with animals or patients. Perfect confounding implies that the corresponding design matrix is not full rank and the model underdetermined. This can be circumvented by certain tricks (where replicates are modeled as the interaction of condition and and a replicate index per condition) which essentially regress repplicates to reference replicates. We believe that this is firstly undesirable as the condition coefficients depend on the identity of the reference replicates and accordingly on the ordering of the replicates, which has no experiental meaning and is purely a result of sample labels. Secondly, such tricks may be hard to come up with in hard cases. Here, we show how one can solve both problems by constraining parameterse in the model. 

For more examples refer to the constraint tutorial in batchglm (~/tutorials/nb_glm_constraints.ipynb).

# Example 1

In this example, we have 4 biological replicates (animals, patients, cell culture replicates etc.) in a treatment experiment: 2 in each condition (treated, untreated). Accordingly, there is perfect confounding at this level. We circumvent this by constraining the biological replicate coefficients. 

## Simulate data:

### Define design matrices for simulation

Here, we built a one-hot encoded design matrix of discrete covariated (biological replicate and treatment) as a numpy array cells x parameters of model. We will later use this design matrix both for the location and scale model

In [3]:
ncells = 2000
dmat = np.zeros([ncells, 6])
dmat[:,0] = 1
dmat[:500,1] = 1 # bio rep 1
dmat[500:1000,2] = 1 # bio rep 2
dmat[1000:1500,3] = 1 # bio rep 3
dmat[1500:2000,4] = 1 # bio rep 4
dmat[1000:2000,5] = 1 # condition effect
print(np.unique(dmat, axis=0))

[[1. 0. 0. 0. 1. 1.]
 [1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]]


### Run simulation

Simulate data using the design matrix defined above.

In [4]:
from batchglm.api.models.nb_glm import Simulator

sim = Simulator(num_features=100)
sim.parse_dmat_loc(dmat = dmat)
sim.parse_dmat_scale(dmat = dmat)
mu=1; phi=0.1
sim.generate_params(rand_fn_loc=lambda size: mu + scipy.stats.truncnorm.rvs(-mu / phi, np.infty, scale=phi, size=size))
sim.generate_data()

## Prepare model estimation

### Design matrix for estimation

Build design matrix is pandas data frame in which the column names are coefficient names and the rows are cells in the same order as in your data object.

In [5]:
dmat_est = pd.DataFrame(data=dmat, columns=["p"+str(i) for i in range(dmat.shape[1])])

Use the diffxpy design matrix builder / formatting function de.test.design_matrix() to build a design matrix that can be input into diffxpy from the pandas data frame.

In [6]:
dmat_est_loc = de.test.design_matrix(dmat = dmat_est)
dmat_est_scale = de.test.design_matrix(dmat = dmat_est)

### Constraints for model

Build constraints based on sets of parameters that have to sum to zero. Each of these constraints is enforced by binding one of these parameters to the rest of the set. Such a constraint is encoded by assigning a 1 to each parameter in the set and a -1 to to the dependent parameter. The constraints have to be ordered so that they can be iteratively applied from top to bottom and so that all independent parameters (1s) are defined at each stage: A dependent parameter may depend on another dependent parameter if the other dependent parameter was defined in a constrained that lies before the current constraint.

In [7]:
constraints_loc = np.zeros([2, dmat_est_loc.dims['design_params']])
# Constraint 0: Account for perfect confouding at biological replicate and treatment level 
# by constraining biological replicate coefficients not to produce mean effects across conditions.
constraints_loc[0,3] = -1
constraints_loc[0,4:5] = 1
# Constraint 1: Account for fact that first level of biological replicates was not absorbed into offset.
constraints_loc[1,1] = -1
constraints_loc[1,2:5] = 1

constraints_loc

array([[ 0.,  0.,  0., -1.,  1.,  0.],
       [ 0., -1.,  1.,  1.,  1.,  0.]])

Use the same constraints also for the scale model:

In [8]:
constraints_scale = constraints_loc.copy()

## Run model fitting and differential expression test:

Here, we perform a single-coefficient Wald test on the treatment effect, accounting for all confounders by using the above defined constraints:

In [9]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

import diffxpy.api as de

test = de.test.wald(
    data = sim.X, # count data
    dmat_loc = dmat_est_loc.data_vars['design'],
    dmat_scale = dmat_est_scale.data_vars['design'],
    constraints_loc = constraints_loc,
    constraints_scale = constraints_scale
)

Estimating model...


KeyError: 'design_loc_params'

### Obtain the results

In [ ]:
test.plot_volcano()